웹 서버 로그 데이터를 활용한 악성 요청 탐지 모델 설계 및 학습



[문제 출제 내용]

주어진 웹 서버 로그 데이터(CSV 파일 : web_server_logs_2.csv)를 사용하여 악성 요청(HTTP 상태 코드 400 이상)을 탐지하는 머신러닝 모델을 설계하세요. 

데이터에는 다양한 요청 메서드와 상태 코드, 응답 크기 등이 포함되어 있으며, 이를 기반으로 분류 모델(Logistic Regression)을 학습시키고, 성능을 평가하세요.

주어진 데이터에서 추가적인 속성을 생성하고, 데이터를 학습에 적합한 형태로 전처리한 뒤, 모델의 성능(정확도, 정밀도, 재현율 등)을 분석하세요.





문제 풀이 시 유의해야 하는 사항

  - 제공된 CSV 데이터를 기반으로 적절한 속성을 생성

  - 모델 성능을 정확도, 정밀도, 재현율, F1-Score와 같은 지표로 평가하고, 결과를 해석



기본적인 가이드

  - 데이터 전처리:

   - timestamp에서 요청 시간대(hour)를 추출하여 새로운 속성을 생성

   - method는 범주형 데이터이므로, 원핫 인코딩으로 변환

   - size는 로그 변환을 적용해 데이터 스케일을 조정하고 이상치의 영향을 줄임.

   - status_code는 성공 여부(is_success)와 에러 여부(is_error)를 나타내는 플래그로 변환

  - 모델 학습

   - Logistic Regression을 사용하여 악성 요청 여부를 예측하는 모델을 학습

  - 결과 평가

   - 테스트 데이터에 대해 모델의 예측 성능을 평가

In [54]:
'''
기본적인 가이드

  - 데이터 전처리:

   - timestamp에서 요청 시간대(hour)를 추출하여 새로운 속성을 생성

   - method는 범주형 데이터이므로, 원핫 인코딩으로 변환

   - size는 로그 변환을 적용해 데이터 스케일을 조정하고 이상치의 영향을 줄임.

   - status_code는 성공 여부(is_success)와 에러 여부(is_error)를 나타내는 플래그로 변환

  - 모델 학습

   - Logistic Regression을 사용하여 악성 요청 여부를 예측하는 모델을 학습

  - 결과 평가

   - 테스트 데이터에 대해 모델의 예측 성능을 평가
'''

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [104]:
df = pd.read_csv('web_server_logs_2.csv')

df.info()
print(type(df['ip'][0]))
# timestamp에서 요청 시간대(hour)를 추출하여 새로운 속성을 생성
df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
print(df['status_code'].describe())

# timestamp에서 요청 시간대(hour)를 추출하여 새로운 속성을 생성
df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
print(df['status_code'].describe())

# method는 범주형 데이터이므로, 원핫 인코딩으로 변환
encoder = OneHotEncoder(sparse_output=False)
encoding = encoder.fit_transform(df[['method']])
method_df = pd.DataFrame(encoding, columns=encoder.get_feature_names_out())
df = pd.concat([df,method_df], axis=1)

#status_code는 성공 여부(is_success)와 에러 여부(is_error)를 나타내는 플래그로 변환
df['status_code_succes'] = df['status_code'][df['status_code'] < 300]
df['status_code_succes'][df['status_code'].astype(int) < 300] = 1
df['status_code_succes'] = df['status_code_succes'].fillna(0)

x = df.drop(['timestamp','method','status_code', 'ip'], axis=1)
y = df['status_code_succes']

train_x, test_x, train_y, test_y = train_test_split(x,y, test_size=0.2, random_state=42)

model = LogisticRegression()
# 2. 로지스틱 회귀 모델 학습
model = LogisticRegression(max_iter=10000) # 로지스틱 회귀 모델 생성 (최대반복 횟수 설정)
model.fit(train_x, train_y) # 학습 데이터로 모델 훈련
# 3. 테스트 데이터 예측
y_pred = model.predict(test_x) # 테스트 데이터를 사용하여 결과 예측

print(classification_report(test_y, y_pred))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ip           1500 non-null   object
 1   timestamp    1500 non-null   object
 2   method       1500 non-null   object
 3   status_code  1500 non-null   int64 
 4   size         1500 non-null   int64 
 5   label        1500 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 70.4+ KB
<class 'str'>
count    1500.000000
mean      279.071333
std       108.459168
min       200.000000
25%       200.000000
50%       200.000000
75%       403.000000
max       503.000000
Name: status_code, dtype: float64
count    1500.000000
mean      279.071333
std       108.459168
min       200.000000
25%       200.000000
50%       200.000000
75%       403.000000
max       503.000000
Name: status_code, dtype: float64
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.0

C:\Users\user\AppData\Local\Temp\ipykernel_14920\3037020362.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['status_code_succes'][df['status_code'].astype(int) < 300] = 1
C:\Users\user\AppData\Local\Temp\ipykernel_14920\3037020362.py

In [ ]:
df = pd.read_csv('web_server_logs_2.csv')

df.info()
print(type(df['ip'][0]))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ip           1500 non-null   object
 1   timestamp    1500 non-null   object
 2   method       1500 non-null   object
 3   status_code  1500 non-null   int64 
 4   size         1500 non-null   int64 
 5   label        1500 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 70.4+ KB
<class 'str'>
count    1500.000000
mean      279.071333
std       108.459168
min       200.000000
25%       200.000000
50%       200.000000
75%       403.000000
max       503.000000
Name: status_code, dtype: float64


In [ ]:
# method는 범주형 데이터이므로, 원핫 인코딩으로 변환
encoder = OneHotEncoder(sparse_output=False)
encoding = encoder.fit_transform(df[['method']])
method_df = pd.DataFrame(encoding, columns=encoder.get_feature_names_out())
df = pd.concat([df,method_df], axis=1)
# size는 로그 변환을 적용해 데이터 스케일을 조정하고 이상치의 영향을 줄임.
df['size'] = np.log1p(df['size'])

,ip,timestamp,method,status_code,size,label,hour,status_code_succes,method_DELETE,method_GET,method_OPTIONS,method_POST,method_PUT
0,192.168.1.138,2024-11-30 09:26:01,OPTIONS,301,6.295266,0,9,0.0,0.0,0.0,1.0,0.0,0.0
1,192.168.1.130,2024-11-30 12:00:34,OPTIONS,200,8.564077,0,12,1.0,0.0,0.0,1.0,0.0,0.0
2,192.168.1.75,2024-12-01 00:22:12,POST,200,8.661986,0,0,1.0,0.0,0.0,0.0,1.0,0.0
3,192.168.1.176,2024-11-30 19:33:26,DELETE,200,8.271804,0,19,1.0,1.0,0.0,0.0,0.0,0.0
4,10.0.0.113,2024-11-30 05:37:26,GET,200,8.957511,0,5,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,192.168.1.65,2024-11-30 05:06:59,GET,503,6.870053,1,5,0.0,0.0,1.0,0.0,0.0,0.0
1496,192.168.1.51,2024-11-30 17:12:08,OPTIONS,200,8.779711,0,17,1.0,0.0,0.0,1.0,0.0,0.0
1497,10.0.0.116,2024-11-30 21:50:56,OPTIONS,200,8.257126,0,21,1.0,0.0,0.0,1.0,0.0,0.0
1498,192.168.1.2,2024-11-30 08:23:04,GET,404,8.375399,1,8,0.0,0.0,1.0,0.0,0.0,0.0


In [84]:
# size는 로그 변환을 적용해 데이터 스케일을 조정하고 이상치의 영향을 줄임.
df['size'] = np.log1p(df['size'])
df

,ip,timestamp,method,status_code,size,label,hour
0,192.168.1.138,2024-11-30 09:26:01,OPTIONS,301,6.295266,0,9
1,192.168.1.130,2024-11-30 12:00:34,OPTIONS,200,8.564077,0,12
2,192.168.1.75,2024-12-01 00:22:12,POST,200,8.661986,0,0
3,192.168.1.176,2024-11-30 19:33:26,DELETE,200,8.271804,0,19
4,10.0.0.113,2024-11-30 05:37:26,GET,200,8.957511,0,5
...,...,...,...,...,...,...,...
1495,192.168.1.65,2024-11-30 05:06:59,GET,503,6.870053,1,5
1496,192.168.1.51,2024-11-30 17:12:08,OPTIONS,200,8.779711,0,17
1497,10.0.0.116,2024-11-30 21:50:56,OPTIONS,200,8.257126,0,21
1498,192.168.1.2,2024-11-30 08:23:04,GET,404,8.375399,1,8


In [ ]:
#status_code는 성공 여부(is_success)와 에러 여부(is_error)를 나타내는 플래그로 변환
df['status_code'] = df['status_code'].astype(str)
df['status_code_succes'][df['status_code'].astype(int) < 300] = 'is_success'
df['status_code'][df['status_code'].astype(int) > 300] = 'is_error'
df['status_code']

In [ ]:
#status_code는 성공 여부(is_success)와 에러 여부(is_error)를 나타내는 플래그로 변환
df['status_code_succes'] = df['status_code'][df['status_code'] < 300]
df['status_code_succes'][df['status_code'].astype(int) < 300] = 1
df['status_code_succes'] = df['status_code_succes'].fillna(0)

In [101]:
x = df.drop(['timestamp','method','status_code', 'ip'], axis=1)
y = df['status_code_succes']

train_x, test_x, train_y, test_y = train_test_split(x,y, test_size=0.2, random_state=42)

In [102]:
model = LogisticRegression()
# 2. 로지스틱 회귀 모델 학습
model = LogisticRegression(max_iter=10000) # 로지스틱 회귀 모델 생성 (최대반복 횟수 설정)
model.fit(train_x, train_y) # 학습 데이터로 모델 훈련
# 3. 테스트 데이터 예측
y_pred = model.predict(test_x) # 테스트 데이터를 사용하여 결과 예측


In [103]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       127
         1.0       1.00      1.00      1.00       173

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



In [65]:
print(type(df['status_code'][1]))

<class 'str'>


In [32]:
df['status_code'][df['status_code'] == 200] = 'succes'

print(200 == 2**)

SyntaxError: invalid syntax (494890471.py, line 3)

In [33]:
df

,ip,timestamp,method,status_code,size,label,hour
0,192.168.1.138,2024-11-30 09:26:01,OPTIONS,301,541,0,9
1,192.168.1.130,2024-11-30 12:00:34,OPTIONS,200,5239,0,12
2,192.168.1.75,2024-12-01 00:22:12,POST,200,5778,0,0
3,192.168.1.176,2024-11-30 19:33:26,DELETE,200,3911,0,19
4,10.0.0.113,2024-11-30 05:37:26,GET,200,7765,0,5
...,...,...,...,...,...,...,...
1495,192.168.1.65,2024-11-30 05:06:59,GET,503,962,1,5
1496,192.168.1.51,2024-11-30 17:12:08,OPTIONS,200,6500,0,17
1497,10.0.0.116,2024-11-30 21:50:56,OPTIONS,200,3854,0,21
1498,192.168.1.2,2024-11-30 08:23:04,GET,404,4338,1,8
